In [1]:
from fastembed import TextEmbedding

In [2]:
query = 'I just discovered the course. Can I join now?'

In [3]:
from qdrant_client import QdrantClient, models

In [4]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [5]:
import json

EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [6]:
embedder = TextEmbedding(model_name=model_handle)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

In [7]:
q, = list(embedder.embed(query))

In [8]:
q.min()

np.float64(-0.11726373885183883)

In [9]:
import numpy as np
np.linalg.norm(q)

np.float64(1.0)

In [10]:
q.dot(q)

np.float64(1.0000000000000002)

In [11]:
doc = 'Can I still join the course after the start date?'

In [12]:
q2, = list(embedder.embed(doc))

In [13]:
q.dot(q2)

np.float64(0.9008528895674548)

In [14]:
#Q3. Ranking by cosine

In [15]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [16]:

#solution 1
import pandas as pd

df = pd.DataFrame(documents)
V_text = list(embedder.embed(df.text))
V_text = np.array(V_text)

In [17]:
# Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.
#solution 2
V = []

for doc in documents:
    vector_input =  doc['question'] + ' ' + doc['text']
    q, = list(embedder.embed(vector_input))
    V.append(q)

In [18]:
np.array(V).dot(q)

array([0.76691052, 0.81023566, 0.80029351, 0.79552661, 1.        ])

In [19]:
V_text = list(embedder.embed(df.question + ' ' + df.text))
V_text = np.array(V_text)

similarity = V_text.dot(q)
similarity.argmax()

np.int64(4)

In [26]:
# Q5. Selecting the embedding model

dimention = pd.DataFrame(TextEmbedding.list_supported_models()) # list of all models


In [30]:
#dimention['dim'].min()
dimention[dimention['dim'] == dimention['dim'].min()]

,model,sources,model_file,description,license,size_in_GB,additional_files,dim,tasks
3,BAAI/bge-small-en,"{'hf': 'Qdrant/bge-small-en', 'url': 'https://...",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,0.130,[],384,{}
4,BAAI/bge-small-en-v1.5,"{'hf': 'qdrant/bge-small-en-v1.5-onnx-q', 'url...",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,0.067,[],384,{}
7,snowflake/snowflake-arctic-embed-xs,"{'hf': 'snowflake/snowflake-arctic-embed-xs', ...",onnx/model.onnx,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.090,[],384,{}
8,snowflake/snowflake-arctic-embed-s,"{'hf': 'snowflake/snowflake-arctic-embed-s', '...",onnx/model.onnx,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.130,[],384,{}
14,sentence-transformers/all-MiniLM-L6-v2,"{'hf': 'qdrant/all-MiniLM-L6-v2-onnx', 'url': ...",model.onnx,"Text embeddings, Unimodal (text), English, 256...",apache-2.0,0.090,[],384,{}
26,sentence-transformers/paraphrase-multilingual-...,{'hf': 'qdrant/paraphrase-multilingual-MiniLM-...,model_optimized.onnx,"Text embeddings, Unimodal (text), Multilingual...",apache-2.0,0.220,[],384,{}


In [33]:
small_model = 'BAAI/bge-small-en'
EMBEDDING_DIMENSIONALITY = 384

In [32]:
# Q6. Indexing with qdrant

import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [34]:
#Create a Collection

# Define the collection name
collection_name = "zoomcamp-llm-homework"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

#text = doc['question'] + ' ' + doc['text']

True

In [35]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=small_model)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [36]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [39]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text 
            text=query,
            model=small_model 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [40]:
search(query)

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.87031734, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])